<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">  

<b> <font size='5'>  
ELEC PRICE PREDICTION CAPSTONE:</font>  

<font size='4'>SARIMAX Model for Continuous Targets</font> </b>

<font size='3'>  
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Manu Kalia Project Submission<br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; DSI-7-SF<br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 15-May-2019<br>
</font>

---


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Function-Definitions" data-toc-modified-id="Function-Definitions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Function Definitions</a></span></li><li><span><a href="#Load-Pickles:--Train/Test-Dataframes-&amp;--Scaled-Arrays" data-toc-modified-id="Load-Pickles:--Train/Test-Dataframes-&amp;--Scaled-Arrays-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load Pickles:  Train/Test Dataframes &amp;  Scaled Arrays</a></span></li><li><span><a href="#Gridsearch-to-find-best-MSE-params-for-P,-D,-Q,-and-S" data-toc-modified-id="Gridsearch-to-find-best-MSE-params-for-P,-D,-Q,-and-S-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Gridsearch to find best MSE params for <code>P</code>, <code>D</code>, <code>Q</code>, and <code>S</code></a></span></li><li><span><a href="#Instantiate-and-fit-the-models-with-best-params" data-toc-modified-id="Instantiate-and-fit-the-models-with-best-params-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Instantiate and fit the models with best params</a></span></li></ul></div>

---

## Imports

In [2]:
import pandas as pd
import numpy as np

import pickle
import wget, os
import time
import glob
import pytz

In [3]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# from pandas.plotting import register_matplotlib_converters
# # 
# register_matplotlib_converters()
# %matplotlib inline
# sns.set_style('whitegrid')

In [4]:
# from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX


---

## Function Definitions

In [5]:
# Code written by Joseph Nelson.

# def interpret_dftest(dftest):
#     dfoutput = pd.Series(dftest[0:2], index=['Test Statistic','p-value'])
#     return dfoutput

---

## Load Pickles:  Train/Test Dataframes &  Scaled Arrays

In [6]:
with open('../data/processed/train.pkl', 'rb') as f:
    train = pickle.load(f)
    
with open('../data/processed/test.pkl', 'rb') as f:
    test = pickle.load(f)
    
with open('../data/processed/train_sc.pkl', 'rb') as f:
    train_sc = pickle.load(f)
    
with open('../data/processed/test_sc.pkl', 'rb') as f:
    test_sc = pickle.load(f)

In [7]:
exog_col_for_dam = [train.columns[i] for i, col in enumerate(train.columns) if col != 'dam_price_per_mwh']
exog_col_for_hasp = [train.columns[i] for i, col in enumerate(train.columns) if col != 'hasp_price_per_mwh']

---

## Gridsearch to find best MSE params for `P`, `D`, `Q`, and `S`

(for `DAM` and `HASP` prices as endogenous variables,  
 use the ARIMA parameters for `p`, `d`, & `q` found in previous ARIMA gridsearch...  
 `DAM: 4, 0, 6`  and  `HASP: 6, 0 ,6`)



In [ ]:
# Matt Brem's grid search loop
# Starting MSE and (P, D, Q).

mse = 99 * (10 ** 16)
final_P = 0
final_D = 0
final_Q = 0
final_S = 0

for P in range(3):
    for Q in range(3):
        for D in range(3):
            for S in range(0,24,4):
                try:
                    # Instantiate SARIMA model.
                    sarima = SARIMAX(endog = train['dam_price_per_mwh'],
                                     order = (4, 0, 6),              # (p, d, q)
                                     seasonal_order = (P, D, Q, S),  # (P, D, Q, S)
                                     exog = train[exog_col_for_dam]) 

                    # Fit SARIMA model.
                    model = sarima.fit()

                    # Generate predictions based on training set.
                    preds = model.predict()

                    # Evaluate predictions.
                    print(f'MSE for (4,0,6) x ({P},{D},{Q},{S}) ... {mean_squared_error(train["dam_price_per_mwh"], preds)}')

                    # Save for final report.
                    if mse > mean_squared_error(train['dam_price_per_mwh'], preds):
                        mse = mean_squared_error(train['dam_price_per_mwh'], preds)
                        final_P = P
                        final_D = D
                        final_Q = Q
                        final_S = S

                except:
                    pass

print(f'Our model that minimizes MSE on the training data is the SARIMA(4,0,6) x ({final_P},{final_D},{final_Q},{final_S}).')
print(f'This model MSE = {mse}')

/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has bee

MSE for (4,0,6) x (0,1,0,4) ... 8506022.687898722


/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


MSE for (4,0,6) x (0,1,0,8) ... 329469.79098115885


/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


...

/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


MSE for (4,0,6) x (1,2,1,12) ... 4200908732.587835


/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


MSE for (4,0,6) x (1,2,1,16) ... 698126209.6526535


/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has bee

In [1]:
# Matt Brem's grid search loop
# Starting MSE and (P, D, Q).

mse = 99 * (10 ** 16)
final_P = 0
final_D = 0
final_Q = 0
final_S = 0

for P in range(2):
    for Q in range(2):
        for D in range(2):
            for S in range(0,25,12):
                try:
                    # Instantiate SARIMA model.
                    sarima = SARIMAX(endog = train['hasp_price_per_mwh'],
                                     order = (6, 0, 6),              # (p, d, q)
                                     seasonal_order = (P, D, Q, S),  # (P, D, Q, S)
                                     exog = train[exog_col_for_hasp]) 

                    # Fit SARIMA model.
                    model = sarima.fit()

                    # Generate predictions based on training set.
                    preds = model.predict()

                    # Evaluate predictions.
                    print(f'MSE for (6,0,6) x ({P},{D},{Q},{S}) ... {mean_squared_error(train["hasp_price_per_mwh"], preds)}')

                    # Save for final report.
                    if mse > mean_squared_error(train['hasp_price_per_mwh'], preds):
                        mse = mean_squared_error(train['hasp_price_per_mwh'], preds)
                        final_P = P
                        final_D = D
                        final_Q = Q
                        final_S = S

                except:
                    pass

print(f'Our model that minimizes MSE on the training data is the SARIMA(6,0,6) x ({final_P},{final_D},{final_Q},{final_S}).')
print(f'This model MSE = {mse}')

Our model that minimizes MSE on the training data is the SARIMA(6,0,6) x (0,0,0,0).
This model MSE = 990000000000000000


## Instantiate and fit the models with best params

In [8]:
P = 0
D = 1
Q = 2
S = 24

# Instantiate SARIMA model.
dam_sarima01224 = SARIMAX(endog = train['dam_price_per_mwh'],
                 order = (4, 0, 6),              # (p, d, q)
                 seasonal_order = (P, D, Q, S),  # (P, D, Q, S)
                 exog = train[exog_col_for_dam]) 

# Fit SARIMA model.
dam_mode012024 = dam_sarima01224.fit()

# Generate predictions based on training set.
dam_preds01224 = dam_mode012024.predict()

# Evaluate predictions.
print(f'MSE for (4,0,6) x ({P},{D},{Q},{S}) ... {mean_squared_error(train["dam_price_per_mwh"], dam_preds01224)}')
 


/Users/owner/anaconda3/envs/ds2/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


KeyboardInterrupt: 

In [ ]:
with open('../fitted_models/dam_sarimaxNNNN_model.pkl', 'wb') as f:
    pickle.dump(dam_sarimaxNNNN_model, f)
    
with open('../data/predictions/dam_sarimaxNNNN_preds.pkl', 'wb') as f:
    pickle.dump(dam_sarimaxNNNN_preds, f)

In [ ]:
P = 0
D = 1
Q = 2
S = 24

# Instantiate SARIMA model.
hasp_sarima10024 = SARIMAX(endog = train['hasp_price_per_mwh'],
                 order = (6, 0, 6),              # (p, d, q)
                 seasonal_order = (P, D, Q, S),  # (P, D, Q, S)
                 exog = train[exog_col_for_hasp]) 

# Fit SARIMA model.
hasp_model10024 = hasp_sarima10024.fit()

# Generate predictions based on training set.
hasp_preds10024 = hasp_model10024.predict()

# Evaluate predictions.
print(f'MSE for (6,0,6) x ({P},{D},{Q},{S}) ... {mean_squared_error(train["hasp_price_per_mwh"], hasp_preds10024)}')
 


In [ ]:
with open('../fitted_models/hasp_sarimaxNNNN_model.pkl', 'wb') as f:
    pickle.dump(hasp_sarimaxNNNN_model, f)
    
with open('../data/predictions/hasp_sarimaxNNNN_preds.pkl', 'wb') as f:
    pickle.dump(hasp_sarimaxNNNN_preds, f)